In [1]:
import pandas as pd
import datetime as dt

In [2]:
# import datasets

event_def = pd.read_csv('../Dataset/event_definitions.csv')
fingerhut_combined = pd.read_csv('../Dataset/fingerhut_combined.csv')

In [3]:
event_def

,event_name,journey_id,event_definition_id,milestone_number,stage
0,application_phone_approved,1,15,1.0,Apply for Credit
1,application_phone_declined,1,16,NaN,Apply for Credit
2,application_phone_pending,1,17,NaN,Apply for Credit
3,application_web_approved,1,12,1.0,Apply for Credit
4,application_web_declined,1,13,NaN,Apply for Credit
5,application_web_pending,1,14,NaN,Apply for Credit
6,application_web_submit,1,3,NaN,Apply for Credit
7,application_web_view,1,19,NaN,Apply for Credit
8,account_activitation,1,29,3.0,Credit Account
9,account_fraud_review,1,37,NaN,Credit Account


In [4]:
event_dict = {
    'Apply for Credit': [15, 17, 12, 14, 3, 19],
    'Account Activation': [29],
    'Fraud Review': [37],
    'Promotion and Discover': [1, 2, 9, 10, 22, 23, 20, 21, 24],
    'Downpayment': [27, 26, 8, 25],
    'Shopping': [11, 6, 4, 5],
    'Place Order': [18, 7],
    'Order Shipped': [28]
}

In [5]:
# merge event_def (event_definition_id column) with fingerhut_combined (ed_id column)

fingerhut_combined = pd.merge(fingerhut_combined, event_def, how='left', left_on='ed_id', right_on='event_definition_id')

In [6]:
# min and max values from 'combined_id' column

min_combined_id = fingerhut_combined['combined_id'].min()
max_combined_id = fingerhut_combined['combined_id'].max()

In [7]:
max_combined_id

1665430

In [8]:
# random 1000 numbers from 1 to 1665430

import random

random.seed(0)

random_numbers = random.sample(range(1, 1665430), 1000)

In [28]:
len(random_numbers)

1000

In [10]:
# keep only rows where combined_id is in random_numbers

fingerhut_small = fingerhut_combined[fingerhut_combined['combined_id'].isin(random_numbers)]

#############################################################################################

In [11]:
# groupby combined_id and only keep groups that has at least one instance of ed_id == 29 (account_activation) and does not have any instance of ed_id == 16 (application_phone_declined) or ed_id == 13 (application_web_declined)

fingerhut_active_account = fingerhut_small.groupby('combined_id').filter(lambda x: (x['ed_id'] == 29).any() & (x['ed_id'] != 16).all() & (x['ed_id'] != 13).all())

In [12]:
# only keep 'combined_id', 'event_timestamp', 'event_name_y'

fingerhut_active_account = fingerhut_active_account[['combined_id', 'ed_id']]

In [13]:
fingerhut_active_account

,combined_id,ed_id
216820,6779,19
216821,6779,19
216822,6779,19
216823,6779,19
216824,6779,19
...,...,...
56704687,1663724,29
56704688,1663724,15
56704689,1663724,18
56704690,1663724,27


In [14]:
# Use the event_dict to create a new column called 'event_type' in fingerhut_active_account. The value of 'event_type' should be the key of the event_dict, which has lists of ed_id as values.

fingerhut_active_account['event_type'] = fingerhut_active_account['ed_id'].map({v: k for k, l in event_dict.items() for v in l})

In [15]:
# remove ed_id column

fingerhut_active_account = fingerhut_active_account.drop(columns=['ed_id'])

# remove duplicates

fingerhut_active_account = fingerhut_active_account.drop_duplicates()

In [16]:
fingerhut_active_account

,combined_id,event_type
216820,6779,Apply for Credit
216832,6779,Shopping
216838,6779,Promotion and Discover
216860,6779,Downpayment
216861,6779,Place Order
...,...,...
56704676,1663724,Promotion and Discover
56704687,1663724,Account Activation
56704689,1663724,Place Order
56704690,1663724,Downpayment


In [17]:
# groupby combined_id, and if the last event_type is not 'Order Shipped', then add new row with event_type 'No Order Made', and also add the combined_id to the new row

fingerhut_active_account = fingerhut_active_account.groupby('combined_id').apply(lambda x: x.append({'event_type': 'No Order Made', 'combined_id': x['combined_id'].iloc[0]}, ignore_index=True) if x['event_type'].iloc[-1] != 'Order Shipped' else x)

# reset index

fingerhut_active_account = fingerhut_active_account.reset_index(drop=True)

/var/folders/6m/88dwrhnx7m3cybxwl1p0rtq40000gn/T/ipykernel_3528/1022833914.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fingerhut_active_account = fingerhut_active_account.groupby('combined_id').apply(lambda x: x.append({'event_type': 'No Order Made', 'combined_id': x['combined_id'].iloc[0]}, ignore_index=True) if x['event_type'].iloc[-1] != 'Order Shipped' else x)
/var/folders/6m/88dwrhnx7m3cybxwl1p0rtq40000gn/T/ipykernel_3528/1022833914.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fingerhut_active_account = fingerhut_active_account.groupby('combined_id').apply(lambda x: x.append({'event_type': 'No Order Made', 'combined_id': x['combined_id'].iloc[0]}, ignore_index=True) if x['event_type'].iloc[-1] != 'Order Shipped' else x)
/var/folders/6m/88dwrhnx7m3cybxwl1p0rtq40000gn/T/ipykernel_3528/10228339

In [18]:
# check for NA values

fingerhut_active_account.isna().sum()

combined_id    0
event_type     0
dtype: int64

In [19]:
fingerhut_active_account['sequence'] = fingerhut_active_account.groupby('combined_id').cumcount() + 1

In [20]:
fingerhut_active_account

,combined_id,event_type,sequence
0,6779,Apply for Credit,1
1,6779,Shopping,2
2,6779,Promotion and Discover,3
3,6779,Downpayment,4
4,6779,Place Order,5
...,...,...,...
1536,1663724,Promotion and Discover,3
1537,1663724,Account Activation,4
1538,1663724,Place Order,5
1539,1663724,Downpayment,6


In [21]:
pivot_fingerhut_active_account = fingerhut_active_account.pivot(index='combined_id', columns='sequence', values='event_type').reset_index(inplace=False)

In [22]:
pivot_fingerhut_active_account = pivot_fingerhut_active_account.fillna('')

In [23]:
pivot_fingerhut_active_account[50:100]

sequence,combined_id,1,2,3,4,5,6,7
50,261748,Promotion and Discover,Apply for Credit,Shopping,Downpayment,Place Order,Account Activation,Order Shipped
51,262639,Apply for Credit,Shopping,Promotion and Discover,Place Order,Downpayment,Account Activation,Order Shipped
52,281901,Promotion and Discover,Apply for Credit,Shopping,Account Activation,No Order Made,,
53,289015,Apply for Credit,Shopping,Account Activation,Promotion and Discover,Place Order,Downpayment,Order Shipped
54,298391,Promotion and Discover,Apply for Credit,Shopping,Account Activation,Downpayment,Place Order,Order Shipped
55,300504,Apply for Credit,Promotion and Discover,Shopping,Place Order,Downpayment,Account Activation,Order Shipped
56,306928,Apply for Credit,Shopping,Downpayment,Place Order,Account Activation,Order Shipped,
57,317454,Account Activation,Promotion and Discover,Shopping,Apply for Credit,Downpayment,Place Order,Order Shipped
58,327620,Account Activation,Shopping,Apply for Credit,Place Order,Downpayment,Promotion and Discover,No Order Made
59,328001,Apply for Credit,Shopping,Promotion and Discover,Account Activation,No Order Made,,


237

In [29]:
import pandas as pd
import plotly.graph_objects as go

def generate_sankey_chart_data(df: pd.DataFrame):
    # list of list: each list is the set of nodes in each tier/column
    column_values = [df[col] for col in df.columns]

    # this generates the labels for the sankey by taking all the unique values
    labels = sum([list(node_values.unique()) for node_values in column_values], [])

    # initializes a dict of dicts (one dict per tier)
    link_mappings = {col: {} for col in df.columns}

    # each dict maps a node to a unique number value
    i = 0
    for col, nodes in zip(df.columns, column_values):
        for node in nodes.unique():
            link_mappings[col][node] = i
            i += 1

    # specifying which columns are serving as sources and which as targets
    source_cols = df.columns[:-1]
    target_cols = df.columns[1:]
    links = []

    # loop to create a list of links in the format [((src, tgt), wt), (), ()...]
    for source_col, target_col in zip(source_cols, target_cols):
        for source, target in zip(df[source_col], df[target_col]):
            links.append(
                (
                    link_mappings[source_col][source],
                    link_mappings[target_col][target],
                    1  # Weight is 1 for counting transitions
                )
            )

    # creating a dataframe with 3 columns: source, target, and weight
    df_links = pd.DataFrame(links, columns=["source", "target", "weight"])

    # generating three lists needed for the sankey visual
    sources = df_links["source"]
    targets = df_links["target"]
    weights = df_links["weight"]

    return labels, sources, targets, weights


# Your DataFrame
df = pivot_fingerhut_active_account.iloc[:, 2:]

# Call the generate_sankey_chart_data function
labels, sources, targets, weights = generate_sankey_chart_data(df=df)


# Map colors to labels
label_colors = {
    'Promotion and Discover': 'pink',
    'Apply for Credit': 'green',
    'Shopping': 'blue',
    'Account Activation': 'cyan',
    'Downpayment': 'purple',
    'Place Order': 'yellow',
    'Order Shipped': 'orange',
    'No Order Made': 'red',
    '': 'black'  # Adjust this for labels with empty strings
}

# Create a DataFrame to aggregate weights
df_links_aggregated = pd.DataFrame({'source': sources, 'target': targets, 'weight': weights})

# Aggregate weights for the same source and target pairs
df_links_aggregated = df_links_aggregated.groupby(['source', 'target'], as_index=False).agg({'weight': 'sum'})

# Create the Sankey diagram
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=labels,
        color=[label_colors[label] for label in labels]  # Map colors based on the dictionary
    ),
    link=dict(
        source=df_links_aggregated['source'],
        target=df_links_aggregated['target'],
        value=df_links_aggregated['weight'],
        # Add labels for each link (source to target) with the total counts
        label=[f"Total Counts: {weight}" for weight in df_links_aggregated['weight']]
    )
)])

fig.update_layout(title_text="Journey flow of customers with account activation (237 random customers)", font_size=10)
fig.show()


Key points:

- A large proportion of customers exposed to the promotion and discover stage move onto: Apply for Credit, Account Activation, Downpayment
- A large proportion of customers move onto Downpayment or Place Order after Account Activation, which may indicate that Account Activation is a significant factor to the customer's decision to purchase. This will require further investigation and hypothesis testing. If it holds to be true then we can focus more on increaseing Account Activation rate than other stages.